In [1]:
import os
import datetime
import pandas as pd
import numpy as np
from example_reactiondiffusion import get_rd_data
from sindy_utils import library_size
from training_v2 import train_network
import tensorflow as tf


RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: initialization failed

In [2]:
training_data, validation_data, test_data = get_rd_data()

In [3]:
params = {}

params['input_dim'] = training_data['y1'].size*training_data['y2'].size
params['latent_dim'] = 2
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.01
params['loss_weight_sindy_x'] = 0.5
params['loss_weight_sindy_regularization'] = 0.1

params['activation'] = 'sigmoid'
params['widths'] = [256]

# training parameters
params['epoch_size'] = training_data['t'].size
params['batch_size'] = 1000
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 3001
params['refinement_epochs'] = 1001

In [ ]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'rd_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 7.118495941162109, (0.16129039, 678.1984, 0.15067537, 0.9988337)
   validation loss 7.529788017272949, (0.16121513, 719.27234, 0.15193182, 0.9988337)
decoder loss ratio: 1.018597, decoder SINDy loss  ratio: 1.164120
Epoch 100
   training loss 0.24350601434707642, (0.09269848, 1.2762685, 0.08362685, 0.96231437)
   validation loss 0.2458089292049408, (0.09598545, 1.294826, 0.081287555, 0.96231437)
decoder loss ratio: 0.606460, decoder SINDy loss  ratio: 0.622835
Epoch 200
   training loss 0.20497074723243713, (0.07087415, 0.6482141, 0.06847048, 0.9337923)
   validation loss 0.20698657631874084, (0.074388266, 0.67420655, 0.06495403, 0.9337923)
decoder loss ratio: 0.470003, decoder SINDy loss  ratio: 0.497685


2022-10-10 21:32:54.510263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-10 21:32:54.540716: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
